In [1]:
# 该版本用的最新版的action recrod
#2022-08-26

In [2]:
import pandas as pd
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import datetime

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## 筛选出符合条件的其他胰岛素用药记录

In [3]:
filedir = "/public/hanl/jupyter_dir/ir_recommend_v2/01.action/"
infile = filedir + '/A_IR_win60.csv'

df_ac_win = pd.read_csv(infile)
df_ac_win["starttime"] = pd.to_datetime(df_ac_win["starttime"])
df_ac_win["endtime"] = pd.to_datetime(df_ac_win["endtime"])
df_ac_win["starttime_bf"] =  df_ac_win["starttime"] + datetime.timedelta(hours=-24)
print(df_ac_win.shape)
df_ac_win.head(2)

(266131, 8)


,stay_id,starttime,endtime,amount,rate,interval,traj_id,starttime_bf
0,30001148,2156-08-30 19:15:00,2156-08-30 20:15:00,2.0,2.0,60.0,1,2156-08-29 19:15:00
1,30001148,2156-08-30 20:15:00,2156-08-30 21:15:00,2.0,2.0,60.0,1,2156-08-29 20:15:00


In [4]:
filedir = "/public/hanl/jupyter_dir/ir_recommend_v2/00.data/"
infile = filedir + '/ir_icu_allinsulin.csv'
df_insu = pd.read_csv(infile)
df_insu["starttime"] = pd.to_datetime(df_insu["starttime"], format='%d/%m/%Y %H:%M:%S')
df_insu["endtime"] = pd.to_datetime(df_insu["endtime"], format='%d/%m/%Y %H:%M:%S')
df_insu = df_insu[df_insu["ordercategoryname"] != "01-Drips"] #这步已将静脉输液注射的regular删除掉了

mapping_dict = {
    223258 : "IR_pump", # pump
    223262 : "OS",
    223260 : "L",
    223259 : "M",
    229299 : "OS", 
    223257 : "M",
    223261 : "M",
    229619 : "L" 
               }

df_insu["type"] = df_insu["itemid"].map(mapping_dict)
df_insu = df_insu.loc[:, ["stay_id", "starttime", "type", "amount"]] # "endtime",
df_insu = df_insu.drop_duplicates()

In [5]:
tmp_df2 = pd.DataFrame(columns=["stay_id", "insu_starttime", "insu_type", "insu_amount"]) 
df_out = pd.DataFrame()
for row in df_insu.itertuples():
    tmp_df2.loc[len(tmp_df2)] = [row.stay_id, row.starttime, row.type, row.amount]
    
    id_ = row.stay_id
    tmp_df = df_ac_win[(df_ac_win["starttime_bf"] <= row.starttime) & (df_ac_win["endtime"] >= row.starttime) & (df_ac_win["stay_id"] >= id_)]
    if tmp_df.shape[0] > 0:
        out = pd.merge(tmp_df, tmp_df2, how="left",left_on="stay_id", right_on="stay_id")
        df_out = pd.concat([df_out, out])
df_out= df_out.drop(["starttime_bf"],axis=1)
df_out.to_csv("S_IF_insu_pump.csv", index=False)
# 9h
# Intermediate file 中间文件

## 按照用药前多少小时的用药形式记录用药

In [6]:
df_out = pd.read_csv("S_IF_insu_pump.csv")

df_out["insu_starttime"] = pd.to_datetime(df_out["insu_starttime"])
df_out["starttime"] = pd.to_datetime(df_out["starttime"])
#df_out = df_out.drop(["starttime_bf"],axis=1)

# df_out筛选：L：用药前24h； M：用药前12h； S：用药前8h
df_out["offset_time_h"] = (df_out["starttime"] - df_out["insu_starttime"]).map(lambda x:x/np.timedelta64(1, 'h')).tolist()
df_out.head(2)

,stay_id,starttime,endtime,amount,rate,interval,traj_id,insu_starttime,insu_type,insu_amount,offset_time_h
0,30001148,2156-08-30 19:15:00,2156-08-30 20:15:00,2.0,2.0,60.0,1,2156-08-30 19:15:00,IR_pump,2.0,0.0
1,30001148,2156-08-30 20:15:00,2156-08-30 21:15:00,2.0,2.0,60.0,1,2156-08-30 19:15:00,IR_pump,2.0,1.0


In [7]:
df_p1 = df_out[df_out["insu_type"].isin(["IR_pump","OS"])]
print(df_p1.shape)
df_p1 = df_p1[df_p1["offset_time_h"]<=8]
df_p1["offset_time"] = df_p1["offset_time_h"]*60
df_p1["offset_time_h"] = round(df_p1["offset_time_h"])
print(df_p1.shape)
df_p1['index'] = range(len(df_p1))

tmp_df = df_p1.loc[:,["index","offset_time_h","offset_time"]]
tmp_df = tmp_df.set_index(["index", "offset_time_h"])["offset_time"]
tmp_df = tmp_df.unstack()
tmp_df.rename_axis(columns=None)
tmp_df = tmp_df.reset_index()
tmp_df.columns = ['index', "OS_bt1", "OS_t0", "OS_t1", "OS_t2", "OS_t3", "OS_t4", "OS_t5", "OS_t6", "OS_t7", "OS_t8"]
df_p11t = tmp_df

tmp_df = df_p1.loc[:,["index","offset_time_h","insu_amount"]]
tmp_df = tmp_df.set_index(["index", "offset_time_h"])["insu_amount"]
tmp_df = tmp_df.unstack()
tmp_df.rename_axis(columns=None)
tmp_df = tmp_df.reset_index()
tmp_df.columns = ['index', "OS_bv1", "OS_v0", "OS_v1", "OS_v2", "OS_v3", "OS_v4", "OS_v5", "OS_v6", "OS_v7", "OS_v8"]
df_p11v = tmp_df

df_p11 = df_p1.loc[:,['stay_id', 'starttime', 'endtime', 'amount', 'rate',"index"]]

tmp_df = pd.merge(df_p11t, df_p11v, how="inner", on = "index")
tmp_df = pd.merge(tmp_df, df_p11, how="inner", on = "index")
tmp_df = tmp_df.drop("index", axis=1)

df_p1 = tmp_df

(2648302, 11)
(386463, 12)


In [8]:
df_p2 = df_out[df_out["insu_type"]=="M"]
print(df_p2.shape)
df_p2 = df_p2[df_p2["offset_time_h"]<=12]
print(df_p2.shape)

df_p2["offset_time"] = df_p2["offset_time_h"]*60

df_p2["offset_time_h"] = round(df_p2["offset_time_h"])
df_p2['index'] = range(len(df_p2))

tmp_df = df_p2.loc[:,["index","offset_time_h","offset_time"]]
tmp_df = tmp_df.set_index(["index", "offset_time_h"])["offset_time"]
tmp_df = tmp_df.unstack()
tmp_df.rename_axis(columns=None)
tmp_df = tmp_df.reset_index()

tmp_df.columns = ['index', "M_bt1", "M_t0", "M_t1", "M_t2", "M_t3", "M_t4", "M_t5", "M_t6", "M_t7", "M_t8", "M_t9", "M_t10", "M_t11", "M_t12"]
df_p11t = tmp_df

tmp_df = df_p2.loc[:,["index","offset_time_h","insu_amount"]]
tmp_df = tmp_df.set_index(["index", "offset_time_h"])["insu_amount"]
tmp_df = tmp_df.unstack()
tmp_df.rename_axis(columns=None)
tmp_df = tmp_df.reset_index()
tmp_df.columns = ['index', "M_bv1", "M_v0", "M_v1", "M_v2", "M_v3", "M_v4", "M_v5", "M_v6", "M_v7", "M_v8", "M_v9", "M_v10", "M_v11", "M_v12"]
df_p11v = tmp_df

df_p11 = df_p2.loc[:,['stay_id', 'starttime', 'endtime', 'amount', 'rate',"index"]]

tmp_df = pd.merge(df_p11t, df_p11v, how="inner", on = "index")
tmp_df = pd.merge(tmp_df, df_p11, how="inner", on = "index")
tmp_df = tmp_df.drop("index", axis=1)

df_p2 = tmp_df

(37137, 11)
(3011, 11)


In [9]:
df_p3 = df_out[df_out["insu_type"]=="L"]
print(df_p3.shape)
df_p3 = df_p3[df_p3["offset_time_h"]<=24]
print(df_p3.shape)

df_p3["offset_time"] = df_p3["offset_time_h"]*60
df_p3["offset_time_h"] = round(df_p3["offset_time_h"])
df_p3['index'] = range(len(df_p3))

tmp_df = df_p3.loc[:,["index","offset_time_h","offset_time"]]
tmp_df = tmp_df.set_index(["index", "offset_time_h"])["offset_time"]
tmp_df = tmp_df.unstack()
tmp_df.rename_axis(columns=None)
tmp_df = tmp_df.reset_index()
tmp_df.columns = ['index', 'L_bt1', 'L_t0', 'L_t1', 'L_t2', 'L_t3', 'L_t4', 'L_t5', 'L_t6', 'L_t7', 'L_t8', 'L_t9', 'L_t10', 'L_t11', 'L_t12', 'L_t13', 'L_t14', 'L_t15', 'L_t16', 'L_t17', 'L_t18', 'L_t19', 'L_t20', 'L_t21', 'L_t22', 'L_t23', 'L_t24']
df_p11t = tmp_df

tmp_df = df_p3.loc[:,["index","offset_time_h","insu_amount"]]
tmp_df = tmp_df.set_index(["index", "offset_time_h"])["insu_amount"]
tmp_df = tmp_df.unstack()
tmp_df.rename_axis(columns=None)
tmp_df = tmp_df.reset_index()
tmp_df.columns = ['index', 'L_bv1', 'L_v0', 'L_v1', 'L_v2', 'L_v3', 'L_v4', 'L_v5', 'L_v6', 'L_v7', 'L_v8', 'L_v9', 'L_v10', 'L_v11', 'L_v12', 'L_v13', 'L_v14', 'L_v15', 'L_v16', 'L_v17', 'L_v18', 'L_v19', 'L_v20', 'L_v21', 'L_v22', 'L_v23', 'L_v24']
df_p11v = tmp_df

df_p11 = df_p3.loc[:,['stay_id', 'starttime', 'endtime', 'amount', 'rate',"index"]]

tmp_df = pd.merge(df_p11t, df_p11v, how="inner", on = "index")
tmp_df = pd.merge(tmp_df, df_p11, how="inner", on = "index")
tmp_df = tmp_df.drop("index", axis=1)
df_p3 = tmp_df

df_p4 = df_out[~df_out["insu_type"].isin(["IR_pump","OS","M","L"])]

(266905, 11)
(66655, 11)


In [10]:
print(df_p1.shape, df_p2.shape, df_p3.shape, df_p4.shape)
df_p1= df_p1.drop_duplicates()
df_p2= df_p2.drop_duplicates()
df_p3= df_p3.drop_duplicates()
df_p4= df_p4.drop_duplicates()
print(df_p1.shape, df_p2.shape, df_p3.shape, df_p4.shape)


(386463, 25) (3011, 33) (66655, 57) (525935, 11)
(227079, 25) (2078, 33) (29289, 57) (118585, 11)


## M: 每个小时只有一次M用药
outfile: df_p2a

In [11]:
# 每个小时只有一次M用药
aa = df_p2
col_lst = ['M_bt1', 'M_t0', 'M_t1', 'M_t2', 'M_t3', 'M_t4', 'M_t5', 'M_t6', 'M_t7', 'M_t8', 'M_t9', 'M_t10', 'M_t11', 'M_t12', 'M_bv1', 'M_v0', 'M_v1','M_v2', 'M_v3', 'M_v4', 'M_v5', 'M_v6', 'M_v7', 'M_v8', 'M_v9', 'M_v10','M_v11', 'M_v12']
tmp_df = aa.loc[:,['stay_id', 'starttime','endtime', 'amount', 'rate']]
for col in col_lst:
    bb = aa.groupby(['stay_id', 'starttime','endtime', 'amount', 'rate'])[col].apply(list).to_frame()
    bb = bb.reset_index()
    tmp_df = pd.merge(tmp_df, bb, how="inner", on=['stay_id', 'starttime','endtime', 'amount', 'rate'])
df_p2a = tmp_df
print(df_p2.shape, df_p2a.shape)
df_p2.index = pd.RangeIndex(len(df_p2.index))

fea_names = ['M_bt1', 'M_t0',
       'M_t1', 'M_t2', 'M_t3', 'M_t4', 'M_t5', 'M_t6', 'M_t7', 'M_t8', 'M_t9',
       'M_t10', 'M_t11', 'M_t12', 'M_bv1', 'M_v0', 'M_v1', 'M_v2', 'M_v3',
       'M_v4', 'M_v5', 'M_v6', 'M_v7', 'M_v8', 'M_v9', 'M_v10', 'M_v11',
       'M_v12']
#tmp_df = df_p2a.loc[:,fea_names].applymap(lambda x: list(set(np.array(x)[~ np.isnan(np.array(x))])))  #[float(str(i).replace("nan","")) for i in x ])
#tmp_df.loc[:,fea_names].applymap(lambda x: len(list(set(x)))) 发现M用药每小时最多一次用药

tmp_df = df_p2a.loc[:,fea_names].applymap(lambda x: max(list(set(x))))

tmp_df2 = df_p2.loc[:,['stay_id', 'starttime', 'endtime', 'amount', 'rate']]
df_p2a = pd.merge(tmp_df, tmp_df2, left_index=True, right_index=True)

(2078, 33) (2078, 33)


In [12]:
## stat
# 用于查看每小时用药最多有几次
#aa = tmp_df.loc[:,fea_names].applymap(lambda x: len(list(set(np.array(x)[~ np.isnan(np.array(x))]))))
fea_names = ['M_bt1', 'M_t0',
       'M_t1', 'M_t2', 'M_t3', 'M_t4', 'M_t5', 'M_t6', 'M_t7', 'M_t8', 'M_t9',
       'M_t10', 'M_t11', 'M_t12', 'M_bv1', 'M_v0', 'M_v1', 'M_v2', 'M_v3',
       'M_v4', 'M_v5', 'M_v6', 'M_v7', 'M_v8', 'M_v9', 'M_v10', 'M_v11',
       'M_v12']
tmp_df = df_p2a.loc[:,fea_names].applymap(lambda x: np.unique(np.array(x)).tolist())

tmp_df2 = tmp_df.loc[:,fea_names].applymap(lambda x: len(np.array(x)[~ np.isnan(np.array(x))]))
tmp_df22 = tmp_df.loc[:,fea_names].applymap(lambda x: np.array(x)[~ np.isnan(np.array(x))])

out_lst = []
for i in fea_names:
    df_outf = pd.DataFrame()
    lst = tmp_df2[tmp_df2[i]> 1].index
    if len(lst) > 0:
        out_lst.append(i)
     #   print(i, lst)
str(out_lst)
# 此步确认只有每小时最多只有一次用药

'[]'

## OS 每小时最多3次用药
outfile: df_p1af

In [13]:
aa = df_p1
col_lst = ['OS_bt1', 'OS_t0', 'OS_t1', 'OS_t2', 'OS_t3', 'OS_t4', 'OS_t5',
       'OS_t6', 'OS_t7', 'OS_t8', 'OS_bv1', 'OS_v0', 'OS_v1', 'OS_v2', 'OS_v3',
       'OS_v4', 'OS_v5', 'OS_v6', 'OS_v7', 'OS_v8']
tmp_df = aa.loc[:,['stay_id', 'starttime','endtime', 'amount', 'rate']]
for col in col_lst:
    bb = aa.groupby(['stay_id', 'starttime','endtime', 'amount', 'rate'])[col].apply(list).to_frame()
    bb = bb.reset_index()
    tmp_df = pd.merge(tmp_df, bb, how="inner", on=['stay_id', 'starttime','endtime', 'amount', 'rate'])
df_p1a = tmp_df
print(df_p1.shape, df_p1a.shape)

fea_names = ['OS_bt1', 'OS_t0', 'OS_t1', 'OS_t2', 'OS_t3', 'OS_t4', 'OS_t5',
       'OS_t6', 'OS_t7', 'OS_t8', 'OS_bv1', 'OS_v0', 'OS_v1', 'OS_v2', 'OS_v3',
       'OS_v4', 'OS_v5', 'OS_v6', 'OS_v7', 'OS_v8']
tmp_df = df_p1a.loc[:,fea_names].applymap(lambda x: np.unique(np.array(x)).tolist())

tmp_df2 = tmp_df.loc[:,fea_names].applymap(lambda x: len(np.array(x)[~ np.isnan(np.array(x))]))
tmp_df22 = tmp_df.loc[:,fea_names].applymap(lambda x: np.array(x)[~ np.isnan(np.array(x))])

#lst22 = []
df_outf2 = pd.DataFrame()
for i in fea_names:
    df_outf = pd.DataFrame()
    #lst = tmp_df2[tmp_df2[i]>1].index
   # if len(lst) > 0:
       # print(i, len(lst))
    #    lst22.extend(lst.tolist())
    new_names = [i + "_" + str(x) for x in range(3)]
    df_outf[new_names[0]], df_outf[new_names[1]], df_outf[new_names[2]] = tmp_df22[i].str[0], tmp_df22[i].str[1], tmp_df22[i].str[-1] 
    df_outf2 = pd.concat([df_outf2, df_outf],axis=1) 
df_outf2.shape

tmp_df = df_p1a.loc[:,['stay_id', 'starttime', 'endtime', 'amount', 'rate']]
df_p1af = pd.merge(tmp_df, df_outf2, left_index=True, right_index=True)
# df_p1af is the output

(227079, 25) (227079, 25)


In [14]:
df_p1a.shape, df_p1af.shape

((227079, 25), (227079, 65))

In [15]:
# stat
fea_names = ['OS_bt1', 'OS_t0', 'OS_t1', 'OS_t2', 'OS_t3', 'OS_t4', 'OS_t5',
       'OS_t6', 'OS_t7', 'OS_t8', 'OS_bv1', 'OS_v0', 'OS_v1', 'OS_v2', 'OS_v3',
       'OS_v4', 'OS_v5', 'OS_v6', 'OS_v7', 'OS_v8']
tmp_df = df_p1a.loc[:,fea_names].applymap(lambda x: np.unique(np.array(x)).tolist())

tmp_df2 = tmp_df.loc[:,fea_names].applymap(lambda x: len(np.array(x)[~ np.isnan(np.array(x))]))
tmp_df22 = tmp_df.loc[:,fea_names].applymap(lambda x: np.array(x)[~ np.isnan(np.array(x))])

out_lst = []
for i in fea_names:
    df_outf = pd.DataFrame()
    lst = tmp_df2[tmp_df2[i]> 4].index
    if len(lst) > 0:
        out_lst.append(i)
     #   print(i, lst)
str(out_lst)

'[]'

## L： 每小时最多2次L用药
outfile：df_p3af

In [16]:
aa = df_p3
col_lst = ['L_bt1', 'L_t0', 'L_t1', 'L_t2', 'L_t3', 'L_t4', 'L_t5', 'L_t6', 'L_t7',
       'L_t8', 'L_t9', 'L_t10', 'L_t11', 'L_t12', 'L_t13', 'L_t14', 'L_t15',
       'L_t16', 'L_t17', 'L_t18', 'L_t19', 'L_t20', 'L_t21', 'L_t22', 'L_t23',
       'L_t24', 'L_bv1', 'L_v0', 'L_v1', 'L_v2', 'L_v3', 'L_v4', 'L_v5',
       'L_v6', 'L_v7', 'L_v8', 'L_v9', 'L_v10', 'L_v11', 'L_v12', 'L_v13',
       'L_v14', 'L_v15', 'L_v16', 'L_v17', 'L_v18', 'L_v19', 'L_v20', 'L_v21',
       'L_v22', 'L_v23', 'L_v24']
tmp_df = aa.loc[:,['stay_id', 'starttime','endtime', 'amount', 'rate']]
for col in col_lst:
    bb = aa.groupby(['stay_id', 'starttime','endtime', 'amount', 'rate'])[col].apply(list).to_frame()
    bb = bb.reset_index()
    tmp_df = pd.merge(tmp_df, bb, how="inner", on=['stay_id', 'starttime','endtime', 'amount', 'rate'])
df_p3a = tmp_df
print(df_p3.shape, df_p3a.shape)

fea_names = ['L_bt1', 'L_t0', 'L_t1', 'L_t2', 'L_t3', 'L_t4', 'L_t5', 'L_t6', 'L_t7',
       'L_t8', 'L_t9', 'L_t10', 'L_t11', 'L_t12', 'L_t13', 'L_t14', 'L_t15',
       'L_t16', 'L_t17', 'L_t18', 'L_t19', 'L_t20', 'L_t21', 'L_t22', 'L_t23',
       'L_t24', 'L_bv1', 'L_v0', 'L_v1', 'L_v2', 'L_v3', 'L_v4', 'L_v5',
       'L_v6', 'L_v7', 'L_v8', 'L_v9', 'L_v10', 'L_v11', 'L_v12', 'L_v13',
       'L_v14', 'L_v15', 'L_v16', 'L_v17', 'L_v18', 'L_v19', 'L_v20', 'L_v21',
       'L_v22', 'L_v23', 'L_v24']
#tmp_df = df_p3a.loc[:,fea_names].applymap(lambda x: np.array(x)[~ np.isnan(np.array(x))])#[float(str(i).replace("nan","")) for i in x ])
tmp_df = df_p3a.loc[:,fea_names].applymap(lambda x: np.unique(np.array(x)).tolist())

tmp_df2 = tmp_df.loc[:,fea_names].applymap(lambda x: len(np.array(x)[~ np.isnan(np.array(x))]))
tmp_df22 = tmp_df.loc[:,fea_names].applymap(lambda x: np.array(x)[~ np.isnan(np.array(x))])

#lst22 = []
df_outf2 = pd.DataFrame()
for i in fea_names:
    df_outf = pd.DataFrame()
    #lst = tmp_df2[tmp_df2[i]>1].index
   # if len(lst) > 0:
       # print(i, len(lst))
    #    lst22.extend(lst.tolist())
    new_names = [i + "_" + str(x) for x in range(2)]
    df_outf[new_names[0]], df_outf[new_names[1]] = tmp_df22[i].str[0], tmp_df22[i].str[-1]
    df_outf2 = pd.concat([df_outf2, df_outf],axis=1) 
df_outf2.shape

tmp_df = df_p3a.loc[:,['stay_id', 'starttime', 'endtime', 'amount', 'rate']]
df_p3af = pd.merge(tmp_df, df_outf2, left_index=True, right_index=True)
# df_p3af is the output

(29289, 57) (29289, 57)


In [17]:
# stat
'''fea_names = ['L_bt1', 'L_t0', 'L_t1', 'L_t2', 'L_t3', 'L_t4', 'L_t5', 'L_t6', 'L_t7',
       'L_t8', 'L_t9', 'L_t10', 'L_t11', 'L_t12', 'L_t13', 'L_t14', 'L_t15',
       'L_t16', 'L_t17', 'L_t18', 'L_t19', 'L_t20', 'L_t21', 'L_t22', 'L_t23',
       'L_t24', 'L_bv1', 'L_v0', 'L_v1', 'L_v2', 'L_v3', 'L_v4', 'L_v5',
       'L_v6', 'L_v7', 'L_v8', 'L_v9', 'L_v10', 'L_v11', 'L_v12', 'L_v13',
       'L_v14', 'L_v15', 'L_v16', 'L_v17', 'L_v18', 'L_v19', 'L_v20', 'L_v21',
       'L_v22', 'L_v23', 'L_v24']
#tmp_df = df_p3a.loc[:,fea_names].applymap(lambda x: np.array(x)[~ np.isnan(np.array(x))])#[float(str(i).replace("nan","")) for i in x ])
tmp_df = df_p3a.loc[:,fea_names].applymap(lambda x: np.unique(np.array(x)).tolist())

tmp_df2 = tmp_df.loc[:,fea_names].applymap(lambda x: len(np.array(x)[~ np.isnan(np.array(x))]))
tmp_df22 = tmp_df.loc[:,fea_names].applymap(lambda x: np.array(x)[~ np.isnan(np.array(x))])
'''
out_lst = []
for i in fea_names:
    df_outf = pd.DataFrame()
    lst = tmp_df2[tmp_df2[i]> 1].index
    if len(lst) > 0:
        out_lst.append(i)
     #   print(i, lst)
str(out_lst)

"['L_bt1', 'L_t0', 'L_t1', 'L_t3', 'L_t4', 'L_t5', 'L_t6', 'L_t7', 'L_t8', 'L_t9', 'L_t10', 'L_t11', 'L_t12', 'L_t13', 'L_t14', 'L_t15', 'L_t16', 'L_t22', 'L_bv1', 'L_v0', 'L_v1', 'L_v16', 'L_v22']"

## merge
IV insulin regular: 342245

In [18]:
df_p3af.shape, df_p1af.shape, df_p2a.shape, df_p4.shape

((29289, 109), (227079, 65), (2078, 33), (118585, 11))

In [19]:
df_p4 = df_p4.loc[:,['stay_id', 'starttime', 'endtime', 'amount', 'rate']]

tmp_df = pd.merge(df_p1af, df_p4,how="outer",on= ['stay_id', 'starttime', 'endtime', 'amount', 'rate']).drop_duplicates()
tmp_df = pd.merge(tmp_df, df_p3af,how="outer",on= ['stay_id', 'starttime', 'endtime', 'amount', 'rate']).drop_duplicates()
tmp_df = pd.merge(tmp_df, df_p2a,how="outer",on= ['stay_id', 'starttime', 'endtime', 'amount', 'rate']).drop_duplicates()
df_insu_pump = tmp_df.fillna(0)

tmp_df.shape

(197023, 197)

In [20]:
df_insu_pump.to_csv("S_insu_pump.csv", index=False)

## def

In [21]:
# 用于查看每小时用药最多有几次
#aa = tmp_df.loc[:,fea_names].applymap(lambda x: len(list(set(np.array(x)[~ np.isnan(np.array(x))]))))
fea_names = ['M_bt1', 'M_t0',
       'M_t1', 'M_t2', 'M_t3', 'M_t4', 'M_t5', 'M_t6', 'M_t7', 'M_t8', 'M_t9',
       'M_t10', 'M_t11', 'M_t12', 'M_bv1', 'M_v0', 'M_v1', 'M_v2', 'M_v3',
       'M_v4', 'M_v5', 'M_v6', 'M_v7', 'M_v8', 'M_v9', 'M_v10', 'M_v11',
       'M_v12']
tmp_df = df_p2a.loc[:,fea_names].applymap(lambda x: np.unique(np.array(x)).tolist())

tmp_df2 = tmp_df.loc[:,fea_names].applymap(lambda x: len(np.array(x)[~ np.isnan(np.array(x))]))
tmp_df22 = tmp_df.loc[:,fea_names].applymap(lambda x: np.array(x)[~ np.isnan(np.array(x))])

out_lst = []
for i in fea_names:
    df_outf = pd.DataFrame()
    lst = tmp_df2[tmp_df2[i]> 1].index
    if len(lst) > 0:
        out_lst.append(i)
     #   print(i, lst)
str(out_lst)
# 此步确认只有每小时最多只有一次用药

'[]'

In [22]:
out_lst = []
for i in fea_names:
    df_outf = pd.DataFrame()
    lst = tmp_df2[tmp_df2[i]> 1].index
    if len(lst) > 0:
        out_lst.append(i)
     #   print(i, lst)
str(out_lst)

'[]'

In [23]:
# 查看前后划分情况
tmp_df22.loc[out_lst]

,M_bt1,M_t0,M_t1,M_t2,M_t3,M_t4,M_t5,M_t6,M_t7,M_t8,M_t9,M_t10,M_t11,M_t12,M_bv1,M_v0,M_v1,M_v2,M_v3,M_v4,M_v5,M_v6,M_v7,M_v8,M_v9,M_v10,M_v11,M_v12


In [24]:
df_p1af.loc[out_lst]

,stay_id,starttime,endtime,amount,rate,OS_bt1_0,OS_bt1_1,OS_bt1_2,OS_t0_0,OS_t0_1,OS_t0_2,OS_t1_0,OS_t1_1,OS_t1_2,OS_t2_0,OS_t2_1,OS_t2_2,OS_t3_0,OS_t3_1,OS_t3_2,OS_t4_0,OS_t4_1,OS_t4_2,OS_t5_0,OS_t5_1,OS_t5_2,OS_t6_0,OS_t6_1,OS_t6_2,OS_t7_0,OS_t7_1,OS_t7_2,OS_t8_0,OS_t8_1,OS_t8_2,OS_bv1_0,OS_bv1_1,OS_bv1_2,OS_v0_0,OS_v0_1,OS_v0_2,OS_v1_0,OS_v1_1,OS_v1_2,OS_v2_0,OS_v2_1,OS_v2_2,OS_v3_0,OS_v3_1,OS_v3_2,OS_v4_0,OS_v4_1,OS_v4_2,OS_v5_0,OS_v5_1,OS_v5_2,OS_v6_0,OS_v6_1,OS_v6_2,OS_v7_0,OS_v7_1,OS_v7_2,OS_v8_0,OS_v8_1,OS_v8_2
